In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *
from vis3d_utilities import *
from metadata import *
from data_manager import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

Setting environment for Local Macbook Pro


In [2]:
all_structures = [u'ExMC_L',
 u'MPul_L',
 u'LPul_L',
 u'IPul_L',
 u'InMC_L',
 u'ExPC_L',
 u'IPulCM_L',
 u'IPulM_L',
 u'InPC_L',
 u'IPulP_L',
 u'DLG_L',
 u'IPulCL_L',
 u'APul_L']

In [15]:
structure_to_color = dict(zip(all_structures, np.array(random_colors(len(all_structures)))/255.))

In [100]:
stack = 'm919LGN'

In [101]:
stack_spec = dict(name=stack, vol_type='annotationAsScore', resolution='10.0um')

In [102]:
structures = DataManager.load_original_volume_all_known_structures_v3(stack_spec=stack_spec, in_bbox_wrt='wholebrain', structures=all_structures)

Prior structure/index map not found. Generating a new one.


In [94]:
polydata_all_structures = {name: volume_to_polydata(structures[name], num_simplify_iter=0, smooth=False)
for name in all_structures}

marching cube: 0.31 seconds
mesh_to_polydata: 0.10 seconds
marching cube: 1.48 seconds
mesh_to_polydata: 0.25 seconds
marching cube: 2.18 seconds
mesh_to_polydata: 0.33 seconds
marching cube: 0.47 seconds
mesh_to_polydata: 0.07 seconds
marching cube: 0.40 seconds
mesh_to_polydata: 0.16 seconds
marching cube: 1.96 seconds
mesh_to_polydata: 0.55 seconds
marching cube: 0.67 seconds
mesh_to_polydata: 0.18 seconds
marching cube: 0.41 seconds
mesh_to_polydata: 0.12 seconds
marching cube: 0.85 seconds
mesh_to_polydata: 0.27 seconds
marching cube: 0.09 seconds
mesh_to_polydata: 0.03 seconds
marching cube: 1.99 seconds
mesh_to_polydata: 0.32 seconds
marching cube: 0.22 seconds
mesh_to_polydata: 0.08 seconds
marching cube: 0.28 seconds
mesh_to_polydata: 0.10 seconds


In [79]:
for name in all_structures:
    output_fp = os.path.join('/home/yuncong/LGN3Datlas/meshes/', stack, name + '.stl')
    create_parent_dir_if_not_exists(output_fp)
    save_mesh_stl(polydata_all_structures[name], output_fp)

In [103]:
c = np.mean([get_centroid_3d(v) + o for name, (v, o) in structures.iteritems()], axis=0)

In [104]:
polydata_centered_all_structures = {name: volume_to_polydata((structures[name][0], structures[name][1]-c), num_simplify_iter=0, smooth=False)
for name in all_structures}

marching cube: 0.47 seconds
mesh_to_polydata: 0.08 seconds
marching cube: 1.77 seconds
mesh_to_polydata: 0.30 seconds
marching cube: 2.23 seconds
mesh_to_polydata: 0.47 seconds
marching cube: 1.62 seconds
mesh_to_polydata: 0.40 seconds
marching cube: 1.02 seconds
mesh_to_polydata: 0.22 seconds
marching cube: 1.59 seconds
mesh_to_polydata: 0.34 seconds
marching cube: 0.44 seconds
mesh_to_polydata: 0.11 seconds
marching cube: 0.34 seconds
mesh_to_polydata: 0.08 seconds
marching cube: 0.88 seconds
mesh_to_polydata: 0.25 seconds
marching cube: 0.09 seconds
mesh_to_polydata: 0.03 seconds
marching cube: 2.09 seconds
mesh_to_polydata: 0.43 seconds
marching cube: 0.29 seconds
mesh_to_polydata: 0.06 seconds
marching cube: 0.07 seconds
mesh_to_polydata: 0.03 seconds


In [105]:
actors_all_structures = {name: actor_mesh(polydata_centered_all_structures[name], 
                                          color=structure_to_color[name], opacity=0.3)
                         for name in all_structures}
#                          for name in ['LPul_L']}

In [106]:
movie_fp = os.path.join('/home/yuncong/LGN3Datlas/animation/', stack + '.avi')
create_parent_dir_if_not_exists(movie_fp)

In [107]:
launch_vtk(actors_all_structures.values(), 
           view_up=(0.000000, -1.000000, 0.000000),
           focal=(0.000000, 0.000000, 1.000000),
            position=(376.799316, 0.000000, -807.048740),
           animate=True, 
           movie_fp=movie_fp)

rm /tmp/brain_video/*
0
/home/yuncong/ffmpeg-3.4.1-64bit-static/ffmpeg -framerate 10 -pattern_type glob -i "/tmp/brain_video/*.png" -c:v libx264 -vf "scale=-1:1080,format=yuv420p" /home/yuncong/LGN3Datlas/animation/m919LGN.avi
0


In [68]:
launch_vtk([actor_mesh(volume_to_polydata(structures['DLG_L'], num_simplify_iter=0, smooth=False))])

marching cube: 1.14 seconds
mesh_to_polydata: 0.29 seconds


# Export all visualizations

In [ ]:
for stack in ['LM30new']:
# for stack in experiments.keys():
    
#     for view in ['coronal', 'horizontal_topDown', 'sagittal', '45']:
    for view in ['sagittal']:

        snapshot_fp = '/home/yuncong/Dropbox/BrainProjectFiguresByTopic/ProjectionLabeling/lauren_data_v2/%(view)s/%(stack)s_%(view)s.png' % {'stack':stack, 'view':view}
        create_parent_dir_if_not_exists(snapshot_fp)

        launch_vtk(
            atlas_structure_actors_down32.values() + \
#             markers_rel2atlas_actors[stack] + \
            [shell_actor_down32_wrt_canonicalAtlasSpace],
            background_color=(1,1,1), 
            axes_label_color=(0,0,0), 
            init_angle=view,
#                         interactive=False,
#                 snapshot_fn=snapshot_fp,
#             snapshot_magnification=2,
            view_up=(-0.012255, -0.999613, 0.024961),
            focal=(-165.374790, -21.539714, 3.094653),
            position= (-187.619574, -80.180217, -2356.163135),
        )
    